In [1]:
import pandas as pd

Loading of the cleaned Datasets

In [2]:
micro = pd.read_csv("data/cleaned/clean_micro_accidents.csv")
ra19 = pd.read_csv("data/cleaned/clean_ra2019.csv")
ra20 = pd.read_csv("data/cleaned/clean_ra2020.csv")
summary = pd.read_csv("data/cleaned/clean_national_summary.csv")

State Accident index

In [3]:
state_risk = micro.groupby("state_name").agg(
    total_accidents=("state_name", "count"),
    total_casualties=("number_of_casualties", "sum")
).reset_index()

state_risk["risk_index"] = (
    state_risk["total_casualties"] / state_risk["total_accidents"]
).round(2)

state_risk.to_csv("data/cleaned/state_risk_index.csv", index=False)


In [4]:
vehicle_risk = micro.groupby("vehicle_type_involved").agg(
    total_accidents=("vehicle_type_involved", "count"),
    total_casualties=("number_of_casualties", "sum")
).reset_index()

vehicle_risk["risk_score"] = (
    vehicle_risk["total_casualties"] / vehicle_risk["total_accidents"]
).round(2)

vehicle_risk.to_csv("data/cleaned/vehicle_risk_index.csv", index=False)


Monthly trend creation

In [7]:
micro.columns


Index(['state_name', 'city_name', 'year', 'month', 'day_of_week',
       'time_of_day', 'accident_severity', 'number_of_vehicles_involved',
       'vehicle_type_involved', 'number_of_casualties', 'number_of_fatalities',
       'weather_conditions', 'road_type', 'road_condition',
       'lighting_conditions', 'traffic_control_presence', 'speed_limit_(km/h)',
       'driver_age', 'driver_gender', 'driver_license_status',
       'alcohol_involvement', 'accident_location_details'],
      dtype='object')

In [8]:
# Combine year & month into proper date
micro["accident_date"] = pd.to_datetime(
    micro["year"].astype(str) + "-" + micro["month"].astype(str) + "-01",
    errors="coerce"
)


In [9]:
monthly_trend = micro.groupby(micro["accident_date"].dt.to_period("M")).size().reset_index(name="accident_count")
monthly_trend["accident_date"] = monthly_trend["accident_date"].astype(str)

monthly_trend.to_csv("data/cleaned/monthly_trend.csv", index=False)


Cost of accidents

In [10]:
COST_PER_CASUALTY = 200000  # Govt avg estimate

micro["estimated_loss"] = micro["number_of_casualties"] * COST_PER_CASUALTY

state_loss = micro.groupby("state_name")["estimated_loss"].sum().reset_index()
state_loss.to_csv("data/cleaned/state_accident_cost.csv", index=False)
